In [ ]:
# install dependencies
!uv sync --all-extras

In [1]:
# load a COCO-pretrained YOLO11n model
from ultralytics import YOLO
model = YOLO("weights/yolo11n.pt")

In [ ]:
# load database from KaggleHub
import kagglehub
path = adilshamim8_rock_paper_scissors_path = kagglehub.dataset_download('adilshamim8/rock-paper-scissors')
print("Path to dataset files:", path)

In [3]:
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import shutil

# 設定原始資料位置
DATA_DIR = os.path.join(path, 'train', 'train')
ANNOTATION_FILE = os.path.join(DATA_DIR, '_annotations.csv')
YOLO_DATASET_DIR = 'data/yolov11_rps_dataset'
os.makedirs(YOLO_DATASET_DIR, exist_ok=True)

In [ ]:
# read annotation CSV
df = pd.read_csv(ANNOTATION_FILE)

# class mapping
class_map = {'Rock': 0, 'Paper': 1, 'Scissors': 2}

# split train/val
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

def convert_and_save(df_split, split_name):
    image_dir = os.path.join(YOLO_DATASET_DIR, 'images', split_name)
    label_dir = os.path.join(YOLO_DATASET_DIR, 'labels', split_name)
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(label_dir, exist_ok=True)

    for _, row in df_split.iterrows():
        filename = row['filename']
        label = row['class']
        xmin, ymin, xmax, ymax = row[['xmin', 'ymin', 'xmax', 'ymax']]

        img_path = os.path.join(DATA_DIR, filename)
        img = Image.open(img_path)
        w, h = img.size

        # 轉為 YOLO 格式 (中心 x/y, 寬, 高，全都除以圖像寬高)
        x_center = (xmin + xmax) / 2 / w
        y_center = (ymin + ymax) / 2 / h
        box_width = (xmax - xmin) / w
        box_height = (ymax - ymin) / h

        label_id = class_map[label]
        yolo_line = f"{label_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}"

        # 複製圖檔 + 儲存標註檔
        shutil.copy(img_path, os.path.join(image_dir, filename))
        with open(os.path.join(label_dir, filename.replace('.jpg', '.txt')), 'a') as f:
            f.write(yolo_line + '\n')

# convert the annotation to yolo format and save train/val splits
convert_and_save(train_df, 'train')
convert_and_save(val_df, 'val')
print("Format conversion complete")

In [ ]:
DATA_YAML_PATH = "data/yolov11_rps_dataset/data.yaml"

# create YOLO training configuration
data_yaml = '''\
train: images/train
val: images/val

nc: 3
names: ['Rock', 'Paper', 'Scissors']
'''
with open(DATA_YAML_PATH, 'w') as f:
    f.write(data_yaml)

print("data.yaml created")

In [ ]:
results = model.train(
    data=DATA_YAML_PATH,
    epochs=25,
    batch=8,
    imgsz=640,
    device="mps",  # remove device="mps" if not using Apple Silicon
    save_period=5, # save checkpoint every 5 epochs
)

In [ ]:
model.export()